<div style="font-family: Arial, Helvetica, sans-serif;">
    <div style="display: flex;padding-top: 20px">
        <div><strong>Course:</strong> Statistical Learning</div>
    </div>
    <div style="display: flex;padding-top: 20px">
        <div style="padding-right: 10px;"><strong>Class:</strong> 22CNTTh</div>
        <div></div>
    </div>
    <div style="display: flex;padding-top: 20px">
        <div style="padding-right: 10px;"><strong>Group:</strong> ...</div>
    </div>
    <div>
        <div style="display: flex;padding-top: 20px">
            <div style="padding-right: 10px;"><strong>Members:</strong></div>
            <div></div>
        </div>
        <table style="font-size: 15px; display:flex;padding-top: 20px">
            <tr>
                <th>No.</th>
                <th>Student ID</th>
                <th>Name</th>
            </tr>
            <tr>
                <td>1</td>
                <td> 22127004 </td>
                <td style="text-align:left;">Trình Cao An</td>
            </tr>
            <tr>
                <td>2</td>
                <td>22127014</td>
                <td style="text-align:left;">Nguyễn Kim Anh</td>
            </tr>
            <tr>
                <td>3</td>
                <td> 22127042 </td>
                <td style="text-align:left;">Võ Thị Kim Khôi</td>
            </tr>
        </table>
    </div>
    <div style="font-size: 25px ;font-weight: 800; text-align: center;padding-top: 20px;">FINAL PROJECT</div>
    <div style="font-size: 20px ;font-weight: 800; text-align: center;padding-top: 20px;">VIETNAMESE STOCKINSIGHT - DATA COLLECTION</div>
</div>

# **I. Import packages**

In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import os
import requests

# **II. Data collection**

## **Articles**

Set up Selenium

In [2]:
def initialize_chrome_driver():
   """
   Initialize Chrome WebDriver with specific configurations
   
   Args:
       user_name (str): Windows username for Chrome profile path
       
   Returns:
       webdriver.Chrome: Configured Chrome WebDriver instance
   """
   # Configure Chrome options
   chrome_options = Options()
   chrome_options.add_argument("--no-sandbox")
   chrome_options.add_argument("--start-maximized")
   
   # Configure download preferences
   chrome_options.add_experimental_option("prefs", {
       "download.default_directory": os.getcwd() + '.\\data',
       "download.prompt_for_download": False,
       "profile.default_content_setting_values.automatic_downloads": 1,
       "profile.default_content_setting_values.popups": 0,
   })
   
   # Initialize and return WebDriver
   return webdriver.Chrome(
       service=Service("D:/Year 3/chromedriver-win64/chromedriver.exe"), 
       options=chrome_options
   )

In [23]:
def get_article_links(driver, date_range, max_pages=100):
    """Collect all article links for the given date range."""
    url = "https://vietstock.vn/chung-khoan.htm"
    driver.get(url)
    # Set date range
    date_input_xpath = '/html/body/div[12]/div[5]/div[1]/div[1]/div[2]/div[1]/div[3]/div[1]/div/input'
    try:
        date_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, date_input_xpath))
        )
        date_input.clear()
        date_input.send_keys(date_range)
        date_input.send_keys(Keys.ENTER)
        time.sleep(2)
    except Exception as e:
        print(f"Failed to set date range: {e}")
        return []
    # Collect links
    article_links = set()
    for page in range(max_pages):
        try:
            print(f"Processing page {page + 1}...")
            soup = BeautifulSoup(driver.page_source, "html.parser")
            for post in soup.select("div.single_post"):
                a = post.select_one("h4 a.fontbold")
                if a and a.get("href"):
                    href = a["href"]
                    if not href.startswith("http"):
                        href = "https://vietstock.vn" + href
                    article_links.add(href)
            # Next page
            next_btn = driver.find_element(By.CSS_SELECTOR, 'ul.pagination li:last-child a')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
            time.sleep(0.5)
            try:
                next_btn.click()
                time.sleep(3)
            except Exception as e:
                driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(3)
        except Exception as e:
            print(f"Stopped at page {page+1}: {e}")
            break
    return list(article_links)

In [24]:
def get_article_details(url):
    """Extract article details from a given URL using requests and BeautifulSoup."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    title = soup.title.string if soup.title else ""
    abstract = ""
    meta = soup.find("meta", property="og:description")
    if meta:
        abstract = meta.get("content", "")
    author = ""
    p_author = soup.find("p", class_="pAuthor")
    if p_author and p_author.a:
        author = p_author.a.text.strip()
    publish_date = ""
    p_time = soup.find("p", class_="pPublishTimeSource")
    if p_time:
        publish_date = p_time.text.strip().replace("-", "").strip()
    content = "\n".join([p.get_text(strip=True) for p in soup.find_all("p", class_="pBody")])
    return {
        "url": url,
        "publish_date": publish_date,
        "author": author,
        "title": title,
        "abstract": abstract,
        "content": content
    }

In [71]:
driver = initialize_chrome_driver()

date_range = "2024-04-01 - 2024-04-30"

links = get_article_links(driver, date_range, max_pages=150)
print(f"Collected {len(links)} article links.")

data = []
for idx, link in enumerate(links):
    try:
        details = get_article_details(link)
        data.append(details)
        print(f"[{idx+1}/{len(links)}] Success: {link}")
    except Exception as e:
        print(f"[{idx+1}/{len(links)}] Failed to get details for {link}: {e}")
 
driver.quit()       
df = pd.DataFrame(data)

Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing page 45...
Processing page 46.

In [72]:
df.to_csv("../data/vietstock17.csv", index=False, encoding="utf-8-sig")

In [6]:
df = pd.read_csv("../data/vietstock01.csv", encoding="utf-8-sig")
print("Load vietstock01.csv")
for i in range (2, 10) :
    df = pd.concat([df, pd.read_csv(f"../data/vietstock0{i}.csv", encoding="utf-8-sig")], ignore_index=True)
    print(f"Load vietstock0{i}.csv")
    
for i in range (10, 21) :
    df = pd.concat([df, pd.read_csv(f"../data/vietstock{i}.csv", encoding="utf-8-sig")], ignore_index=True)
    print(f"Load vietstock{i}.csv")

Load vietstock01.csv
Load vietstock02.csv
Load vietstock03.csv
Load vietstock04.csv
Load vietstock05.csv
Load vietstock06.csv
Load vietstock07.csv
Load vietstock08.csv
Load vietstock09.csv
Load vietstock10.csv
Load vietstock11.csv
Load vietstock12.csv
Load vietstock13.csv
Load vietstock14.csv
Load vietstock15.csv
Load vietstock16.csv
Load vietstock17.csv
Load vietstock18.csv
Load vietstock19.csv
Load vietstock20.csv


In [7]:
df.shape

(7451, 6)

In [ ]:
df.to_csv("../data/vietstock.csv", index=False, encoding="utf-8-sig")

## **Stock codes**

In [ ]:
driver = initialize_chrome_driver()
url = "https://www.tradingview.com/markets/stocks-vietnam/market-movers-all-stocks/"
driver.get(url)
time.sleep(3)

i = 0
while True:
    try:
        # Click bằng JavaScript
        driver.execute_script("""
            var btn = document.querySelector("#js-category-content > div.js-base-screener-page-component-root > div > div.content-vcNgbmvT > div > div.loadMoreWrapper-YZEOoLh1 > button");
            if(btn) btn.click();
        """)
        print(f"Clicked 'Load More' button {i + 1} times")
        i += 1
        time.sleep(2.5)
        # Kiểm tra còn nút không, nếu không thì break
        btn_exists = driver.execute_script("""
            return !!document.querySelector("#js-category-content > div.js-base-screener-page-component-root > div > div.content-vcNgbmvT > div > div.loadMoreWrapper-YZEOoLh1 > button");
        """)
        if not btn_exists:
            print("No more Load More button.")
            break
    except Exception as e:
        print("Error or no more button:", e)
        break

# Lấy tất cả các thẻ <tr> trong tbody
rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr[data-rowkey]")

stock_codes = []
for row in rows:
    data_rowkey = row.get_attribute("data-rowkey")
    if data_rowkey and ":" in data_rowkey:
        code = data_rowkey.split(":")[1]
        stock_codes.append(code)

driver.quit()

Clicked 'Load More' button 1 times
Clicked 'Load More' button 2 times
Clicked 'Load More' button 3 times
Clicked 'Load More' button 4 times
Clicked 'Load More' button 5 times
Clicked 'Load More' button 6 times
Clicked 'Load More' button 7 times
Clicked 'Load More' button 8 times
Clicked 'Load More' button 9 times
Clicked 'Load More' button 10 times
Clicked 'Load More' button 11 times
Clicked 'Load More' button 12 times
Clicked 'Load More' button 13 times
No more Load More button.


In [12]:
stock_codes = list(dict.fromkeys(stock_codes))
print(f"Total stock codes: {len(stock_codes)}")
print(stock_codes)

Total stock codes: 1319
['A32', 'AAA', 'AAH', 'AAM', 'AAS', 'AAT', 'AAV', 'ABB', 'ABC', 'ABI', 'ABR', 'ABS', 'ABT', 'ABW', 'ACB', 'ACC', 'ACE', 'ACG', 'ACL', 'ACM', 'ACS', 'ACV', 'ADC', 'ADG', 'ADP', 'ADS', 'AFX', 'AG1', 'AGF', 'AGG', 'AGM', 'AGP', 'AGR', 'AGX', 'AIC', 'AIG', 'ALT', 'ALV', 'AMC', 'AME', 'AMP', 'AMS', 'ANT', 'ANV', 'APC', 'APF', 'APG', 'APH', 'API', 'APL', 'APP', 'APS', 'ASG', 'ASM', 'ASP', 'AST', 'ATA', 'ATB', 'ATG', 'ATS', 'AVC', 'AVG', 'BAB', 'BAF', 'BAL', 'BAX', 'BBC', 'BBH', 'BBM', 'BBS', 'BBT', 'BCA', 'BCC', 'BCE', 'BCF', 'BCG', 'BCM', 'BCP', 'BCR', 'BDB', 'BDG', 'BDT', 'BDW', 'BED', 'BEL', 'BFC', 'BGE', 'BGW', 'BHA', 'BHH', 'BHI', 'BHN', 'BHP', 'BIC', 'BID', 'BIG', 'BII', 'BIO', 'BKC', 'BKG', 'BLF', 'BLI', 'BLN', 'BLT', 'BMC', 'BMF', 'BMI', 'BMJ', 'BMK', 'BMN', 'BMP', 'BMS', 'BMV', 'BNA', 'BOT', 'BPC', 'BQB', 'BRC', 'BRR', 'BRS', 'BSA', 'BSD', 'BSG', 'BSH', 'BSI', 'BSL', 'BSP', 'BSQ', 'BSR', 'BST', 'BTB', 'BTD', 'BTH', 'BTN', 'BTP', 'BTS', 'BTU', 'BTW', 'BVB', 'B

In [13]:
with open("all_vn_stock_codes.txt", "w", encoding="utf-8") as f:
    for code in stock_codes:
        f.write(code + "\n")